# 1. Power Method
Find the largest eigenvalue [See Here](https://www.youtube.com/watch?v=yBiQh1vsCLU)
* Let A be a Matrix and x0=[1,0,....0]

In [2]:
import numpy as np
def power_method(A,k=10):
    '''
    Return the largest eigenvalue of matrix A with k iteratons
    '''
    m,n=A.shape
    if m!=n: return 'The matrix should be squer'
    x=np.zeros(n)
    x[0]=1.0
    for i in range(k):
        p=np.dot(A,x)
        m=np.max(np.abs(p))
        x=p/m 
        # p/la.norm(p)
    return m

### Example

In [31]:
A=np.array([[7,9],[9,7]])
# From linear algebra we can find that A has two eigenvalue 16 and -2. So the max is 16 and our function estimate it.
power_method(A)

15.999999895691872

# 2.QR Factorization

In [3]:
from scipy.linalg import lu
A = np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]])
P, L, U = lu(A)
print(P)
print(L)
print(U)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[ 1.          0.          0.          0.        ]
 [ 0.28571429  1.          0.          0.        ]
 [ 0.71428571  0.12        1.          0.        ]
 [ 0.71428571 -0.44       -0.46153846  1.        ]]
[[ 7.          5.          6.          6.        ]
 [ 0.          3.57142857  6.28571429  5.28571429]
 [ 0.          0.         -1.04        3.08      ]
 [ 0.          0.          0.          7.46153846]]


# 3. Invers Problem for Eigenvalue
But does **power_method** work for a big dimention matrix?<br>
So lets to check it. Please give me a matrix of dim 50 that its largest eigenvalue is 23.<br>
Is it difficult?<br>
So we want do it by QR factorization.<br>

* **Goal:** Make a matrix such that its eigenvalue are the element of an arbitrary array.

In [4]:
from scipy import linalg as la
def eigenvalue_invers_problem(b):
    '''
    Return Diagonal matrix B related to batrix b
    '''
    n=len(b)
    D=np.diag(b)
    R=np.random.rand(n,n)
    q,r = la.qr(R)
    B=np.dot(np.transpose(q),np.dot(D,q))
    return B

### Example

In [82]:
K=eigenvalue_invers_problem(np.array([12,-2,7]))
power_method(K)

12.035859204704591

### Example

In [86]:
n=7
b=np.linspace(1,n,n)
print(b)
B=eigenvalue_invers_problem(b)
power_method(B)

[1. 2. 3. 4. 5. 6. 7.]


6.675158013863618

# 4. Sparse matrix

## 4.1 What is sparse matrix?

We can make a diagonal matrix with an array

In [5]:
np.diag(np.array([5,8,-6]))

array([[ 5,  0,  0],
       [ 0,  8,  0],
       [ 0,  0, -6]])

## 4.2 Sparse Package

Diagonal matrix is a spesial kind of sparse matrix. Sparse matrix is a matrix, which is almost empty.(Have a lot of zeros).
The is a package in scipy that you can work with them.

In [7]:
import scipy.sparse as spsp
#Run help for more details:
#?spsp

### Example

In [18]:
A = spsp.spdiags([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]], [0, 1], 5, 5)

print(A)
#print(type(A))
#print(spsp.issparse(A))
#print(A.todense())


  (0, 0)	1
  (1, 1)	2
  (2, 2)	3
  (3, 3)	4
  (4, 4)	5
  (0, 1)	7
  (1, 2)	8
  (2, 3)	9
  (3, 4)	10


### Example

In [44]:
N=6
diagonals = np.zeros((4, N))   # 3 diagonals
diagonals[0,:] = -1
diagonals[1,:] = 2
diagonals[2,:] = 3
diagonals[3,:] = 4

diagonals
B = spsp.spdiags(diagonals, [-1,0,1,2], N, N, format='csc')
print(B.todense())

[[ 2.  3.  4.  0.  0.  0.]
 [-1.  2.  3.  4.  0.  0.]
 [ 0. -1.  2.  3.  4.  0.]
 [ 0.  0. -1.  2.  3.  4.]
 [ 0.  0.  0. -1.  2.  3.]
 [ 0.  0.  0.  0. -1.  2.]]


## 4.3 Solve a Sparse Equation

We want to solve an equation that matrix $A$ is sparse
$$
AX=b
$$

### A. Use dsolve

In [45]:
from scipy.sparse.linalg import dsolve

#Let solve BX=b where B is from above Example
b=np.array([2,4,12,5,7,-4])

x=dsolve.spsolve(B, b, use_umfpack=True)
print(x)

[-3.00239521  0.74730539  1.44071856 -1.20479042  3.37005988 -0.31497006]


In [54]:
# Test Solution and Error
print(B*x)
print(f"Error: {np.mean(B * x - b)}")  

[ 2.  4. 12.  5.  7. -4.]
Error: 2.9605947323337506e-16


### B. Use SuperLU

Another method for solve a sparse matrix is its SuperLU 

In [47]:
factor=dsolve.splu(B)

In [48]:
print(factor)

In [56]:
xf=factor.solve(b) ## we use solve insted of dsolve
print(xf)

[-3.00239521  0.74730539  1.44071856 -1.20479042  3.37005988 -0.31497006]


In [55]:
# Test Solution and Error for SuperLU solution
print(B*xf)
print(f"Error: {np.mean(B * xf - b)}")

[ 2.  4. 12.  5.  7. -4.]
Error: 2.9605947323337506e-16
